<a href="https://colab.research.google.com/github/Liqs-v2/octopack-refactoring-evaluation/blob/main/src/refact_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Imports

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

from typing import List

## Load model

In [3]:
checkpoint = "smallcloudai/Refact-1_6B-fim"
device = "cuda" # for GPU usage or "cpu" for CPU usage

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

tokenizer_config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/smallcloudai/Refact-1_6B-fim:
- configuration_gpt_refact.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_gpt_refact.py:   0%|          | 0.00/24.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/smallcloudai/Refact-1_6B-fim:
- modeling_gpt_refact.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Load dataset

In [5]:
dataset = load_dataset("bigcode/humanevalpack", "python")

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

# Evaluate model

## Build queries for prompts

Create a query format matching the one specified in appendix N of the [octopack paper](https://arxiv.org/abs/2308.07124) for `HumanEvalFix`. In Table 16 of appendix N the authors state that "If no function start or no context is present,
that part is not added to the prompt (and the preceding newline is also removed)." I interpreted the horizontal bars in their sample query for `HumanEvalFix` as newlines, thus adding "\n\n" between the "Instruction" and "Context sections as well as the "Context" and "Function start" sections.

In [6]:
sample_query = dataset['test'][0]['instruction'] + "\n\n" + dataset['test'][0]['declaration'] + dataset['test'][0]['buggy_solution'] + "\n" + dataset['test'][0]['declaration']

In [7]:
print(sample_query)

Write a Python function `has_close_elements(numbers: List[float], threshold: float) -> bool` to solve the following problem:
Check if in given list of numbers, are any two numbers closer to each other than
given threshold.
>>> has_close_elements([1.0, 2.0, 3.0], 0.5)
False
>>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
True

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = elem - elem2
                if distance < threshold:
                    return True

    return False

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:



Since the model can accept at most inputs of dimension 4096, I quickly check whether there is a chance that this limit is exceeded, in which case I would need to apply some transformations in order to chunk the input.

In [8]:
declaration_max_length = max(len(s) for s in dataset['test']['declaration'])
buggy_max_length = max(len(s) for s in dataset['test']['buggy_solution'])
instruction_max_length = max(len(s) for s in dataset['test']['instruction'])
assert((declaration_max_length + buggy_max_length + instruction_max_length) < 4096)

## Prompt model for refactoring

As the authors in the [octopack paper](https://arxiv.org/abs/2308.07124) suggest (see appendix N) I "do not optimize prompts and go with the format provided by the respective
model authors or the most intuitive format if none is provided."

Since the [Refact-1.6B model](https://huggingface.co/smallcloudai/Refact-1_6B-fim) provides a prompting template for chat use, which is suggested in the task description, we use this template.

In [9]:
prompt_template = "<empty_output>SYSTEM {system}\n" \
                  "<empty_output>USER {query}\n" \
                  "<empty_output>ASSISTANT"

### Model evaluation utilities
To evaluate our model we perform pairwise comparison between the lines returned by the model and the solution specified in the dataset. To be a bit more lenient, we remove irrelevant lines that just contain whitespaces from both solutions.

In [10]:
def extract_non_empty_lines_from(solution: str) -> List[str]:
  """
  Splits the passed in solution by line break delimiters '\n' and removes
  lines that only contain whitespace.

  Args:
    solution (str): A solution to the bugfix task as raw string.

  Returns:
    (List[str]): The passed in solution split by line break delimiters without empty lines that contain only whitespace.
  """
  solution_lines = solution.split('\n')
  return [line for line in solution_lines if line.strip() != '']

def is_model_solution_correct(model_solution: str, sample_solution: str) -> bool:
  """
  Evaluates the model solution. A model solution is accepted if it is the same length as the canonical solution and
  the pairwise evaluation of the lines at the same indices by string equality succeeds for every line.

  Args:
    model_solution (str): The raw string of the solution generated by the model, without noise or helper tokens generated by the model.
    sample_solution (str): The canonical solution for the bugfix problem with the function declaration prepended to it.

  Returns:
    bool: True if the evaluation as defined above succeeds, False otherwise.
  """
  model_solution_lines = extract_non_empty_lines_from(model_solution)
  sample_solution_lines = extract_non_empty_lines_from(sample_solution)

  #if len(sample_solution_lines) != len(model_solution_lines):
  #  return False

  solution_lines = zip(sample_solution_lines, model_solution_lines)

  for sample_solution_line, model_solution_line in solution_lines:
    if sample_solution_line != model_solution_line:
      print("Solution mismatch detected!")
      print(f"Sample solution: {sample_solution_line}")
      print(f"Model solution: {model_solution_line}")
      return False

  return True

In [11]:
model_solutions = []
model_solution_evaluations = []

for data in dataset['test']:
  instruction = data['instruction']
  context = data['declaration'] + data['buggy_solution']
  function_start = data['declaration']
  query = instruction + "\n\n" + context + "\n" + function_start

  prompt = prompt_template.format(system="You are a programming assistant",
                                query=query)

  inputs = tokenizer.encode_plus(prompt, return_tensors="pt", truncation=True, max_length=4096, return_attention_mask=True)
  input_ids = inputs["input_ids"].to(device)
  attention_mask = inputs["attention_mask"].to(device)

  outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=4096, temperature=0.2)
  result = tokenizer.decode(outputs[0])

  model_solution = (result.split("ASSISTANT ")[-1]).split("<empty_output>")[0]
  sample_solution = data['declaration'] + data['canonical_solution']

  model_solutions.append(model_solution)
  model_solution_evaluations.append(is_model_solution_correct(model_solution, sample_solution))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Solution mismatch detected!
Sample solution:             if idx != idx2:
Model solution:             if idx!= idx2:


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Solution mismatch detected!
Sample solution:             if current_depth == 0:
Model solution:             if current_depth < 0:


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Solution mismatch detected!
Sample solution:         expression+= oprt + str(oprn)
Model solution:         expression+=expression + oprt + str(oprn)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Solution mismatch detected!
Sample solution:             new_str[idx] = i.swapcase()
Model solution:             new_str[idx] = i


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Solution mismatch detected!
Sample solution:     return hashlib.md5(text.encode('ascii')).hexdigest() if text else None
Model solution:     if text:
Solution mismatch detected!
Sample solution:     return [i for i in range(lower, upper+1) if i % 2 == 0]
Model solution:     return [i for i in range(lower, upper) if i % 2 == 0]


In [13]:
correct = 0
for eval in model_solution_evaluations:
  if eval:
    correct +=1

In [16]:
round((correct/len(model_solution_evaluations)) * 100, 2)

7.93

The model only manages to generate the correct solution in 7.93% of the cases, which corresponds to the "Zero-shot pass@1 (%)" reported in section I of the appendix of the [octopack paper](https://arxiv.org/abs/2308.07124). This pales in comparison to the performance (41.7%) of even the worst of the models that were evaluated in this paper.

Looking at the output of the model often times gets close to the canonical solution, but is off due to some small mistake (e.g., forgets to add 1 or is missign one part of an if condition). These are cases where a bigger model might be able to amend the issues, as it might simply not have the capacity to produce in such detail.

I found the following to be a particularly interesting error, indicating the limitations of LLMs based on the data they were trained on.

> Sample solution:     planet_names = ("Mercury", "Venus", "Earth", "Mars", "**Jupiter**", "Saturn", "Uranus", "Neptune")

> Model solution:     planet_names = ("Mercury", "Venus", "Earth", "Mars", "**Jupyter**", "Saturn", "Uranus", "Neptune")

